In [1]:
import gzip
import re

In [2]:
with gzip.open("../data/poczatki_wikipediowe.txt.gz", "rt") as f:
    file_content = f.read()

In [3]:
file_content.count("### ")

1008143

In [4]:
file_content.count("\n") / 3

1008135.6666666666

In [6]:
d = {}
for part in file_content.removeprefix("### ").split("\n### "):
    splited = part.strip().split("\n", maxsplit=1)
    if len(splited) == 2:
        title, content = splited
        d[title] = content

In [17]:
sum(v.startswith("'''") for v in d.values())

561805

In [167]:
d["Wahadłowiec kosmiczny"]

"'''Wahadłowiec kosmiczny''', '''prom kosmiczny''' - rodzaj załogowego '''Wahadłowiec kosmiczny''', '''prom kosmiczny''' - rodzaj załogowego statku kosmicznego, który może być wykorzystywany wielokrotnie i zwykle przystosowany jest do wynoszenia na orbitę i ściągania z orbity sztucznych satelitów i innych ładunków. W założeniach prom, dzięki możliwości"

In [176]:
basic_pattern = re.compile(r"'''([\w\s]+)'''(?:, '''([\w\s]+)''')+")
basic_extract_pattern = re.compile(r"'''(?<!,)([\w\s]+)'''")

In [ ]:
another_pattern = re.compile(r"'''([\w\s]+)'''(?:, '''([\w\s]+)''')+")

In [186]:
with open("out", "wt") as f:
    for k, v in d.items():
        if v.startswith("* "):
            # enumeration
            pass
        elif m := basic_pattern.match(v):
            synonyms = {k}
            synonyms.update(basic_extract_pattern.findall(m.group()))
            f.write(" # ".join(synonyms) + "\n")

In [180]:
idx = file_content.find("Horyzont II")
file_content[idx - 100: idx + 1000]

"ich kolei parowych . W 1748 na odcinku kilku kilometrów z kopalni węgla Wylam do rzeki Tyne\n\n### MS Horyzont II\n'''Horyzont II''', '''Horyzont II''', statek szkolno-badawczy Akademii Morskiej w Gdyni. Zbudowany w Gdańskiej Stoczni Remontowej w latach 1999-2000. Jednostka została zmodyfikowana z zaprojektowanego w 1987 roku przez dr Dariusza Boguckiego żaglowca “Polarex . Miał on pływać na podbiegunowe ekspedycje doświadczalne\n\n### Skurcz betonu\n'''Skurcz betonu''' &amp;ndash; zmiana objętości betonu, niezależna od naprężeń, zachodzi w suchym środowisku. Rozróżniamy skurcz autogeniczny i skurcz spowodowany wysychaniem. Skurcz autogeniczny jest kilkakrotnie mniejszy od skurczu wywołanego wysychaniem. Najważniejszymi czynnikami wpływającymi na skurcz betonu są: klasa betonu, wilgotność środowiska RH, rodzaj\n\n### Doktryna obronna Rzeczypospolitej Polskiej\n'''Doktryna obronna Rzeczypospolitej Polskiej''', przyjęta przez Komitet Obrony Kraju 21 lutego 1990 r., jest dokumentem państ